## Definitions

### Libraries, Devices

In [70]:
import warnings

warnings.filterwarnings("ignore")

import itertools
import os
import random
import sys
import torch.nn.functional as F
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import segmentation_models_pytorch as smp
import torch
import xarray as xr
from torch.utils.data import Dataset
import math
from tqdm import tqdm

from scipy.signal import convolve2d

from cosine_annealing_warmup import CosineAnnealingWarmupRestarts

# from torch.optim.lr_scheduler import ReduceLROnPlateau
from holodecml.transforms import LoadTransformations
from echo.src.base_objective import BaseObjective

# from holodecml.seed import seed_everything
from holodecml.data import XarrayReader
from holodecml.models import load_model
from holodecml.losses import load_loss

from collections import defaultdict
from argparse import ArgumentParser
from pathlib import Path
from torch.utils.data import Dataset


import xarray as xr
import torch.nn.functional as F
import pandas as pd
import numpy as np
import subprocess
import torch.fft
import logging
import shutil
import random
import psutil
import optuna
import torch
import time
import tqdm
import gc
import os
import sys
import itertools
import yaml
import warnings

In [71]:
torch.cuda.empty_cache()

In [72]:
is_cuda = torch.cuda.is_available()
print(is_cuda)
device = torch.device(torch.cuda.current_device()) if is_cuda else torch.device("cpu")
print(device)
if is_cuda:
    torch.backends.cudnn.benchmark = True
    #torch.backends.cudnn.deterministic = True

True
cuda:0


### Filepaths

In [73]:
train_dataset_path = "/glade/p/cisl/aiml/ai4ess_hackathon/holodec/synthetic_holograms_500particle_gamma_4872x3248_training.nc"
test_dataset_path = "/glade/p/cisl/aiml/ai4ess_hackathon/holodec/synthetic_holograms_500particle_gamma_4872x3248_validation.nc"



### Processing Functions

### Loading Functions

In [74]:
class WavePropagator(object):

    def __init__(self,
                 data_path,
                 n_bins=1000,
                 step_size=128,
                 tile_size = 128,
                 marker_size=10,
                 transform_mode=None,
                 device="cpu"):

        self.h_ds = xr.open_dataset(data_path)


        if 'zMin' in self.h_ds.attrs:
            self.zMin = self.h_ds.attrs['zMin']  # minimum z in sample volume
            self.zMax = self.h_ds.attrs['zMax']
        else:  # some of the raw data does not have this parameter
            # should warn the user here through the logger
            self.zMin = 0.014
            self.zMax = 0.158 #15.8 - 1.4 / (1000)

        self.n_bins = n_bins
        self.z_bins = np.linspace(
            self.zMin, self.zMax, n_bins+1)*1e6  # histogram bin edges
        self.z_centers = self.z_bins[:-1] + 0.5 * \
            np.diff(self.z_bins)  # histogram bin center
        
        self.tile_size = tile_size
        self.step_size = step_size
        
        # UNET gaussian marker width (standard deviation) in um
        self.marker_size = marker_size
        self.device = device


        self.dx = self.h_ds.attrs['dx']      # horizontal resolution
        self.dy = self.h_ds.attrs['dy']      # vertical resolution
        self.Nx = int(self.h_ds.attrs['Nx'])  # number of horizontal pixels
        self.Ny = int(self.h_ds.attrs['Ny'])  # number of vertical pixels
        self.lam = self.h_ds.attrs['lambda']  # wavelength
        self.image_norm = 255.0
        self.transform_mode = transform_mode
        self.x_arr = np.arange(-self.Nx//2, self.Nx//2)*self.dx
        self.y_arr = np.arange(-self.Ny//2, self.Ny//2)*self.dy

        self.tile_x_bins = np.arange(-self.Nx//2,
                                     self.Nx//2, self.step_size)*self.dx*1e6
        self.tile_y_bins = np.arange(-self.Ny//2,
                                     self.Ny//2, self.step_size)*self.dy*1e6

        self.fx = torch.fft.fftfreq(
            self.Nx, self.dx, device=self.device).unsqueeze(0).unsqueeze(2)
        self.fy = torch.fft.fftfreq(
            self.Ny, self.dy, device=self.device).unsqueeze(0).unsqueeze(1)

        
        self.create_mapping()
    def torch_holo_set(self,
                       Ein: torch.tensor,
                       z_tnsr: torch.tensor):
        """
        Propagates an electric field a distance z
        Ein complex torch.tensor
        - input electric field

        fx:real torch.tensor
        - x frequency axis (3D, setup to broadcast)

        fy: real torch.tensor
        - y frequency axis (3D, setup to broadcast)

        z_tnsr: torch.tensor
        - tensor of distances to propagate the wave Ein
            expected to have dims (Nz,1,1) where Nz is the number of z
            dimensions

        lam: float
        - wavelength

        returns: complex torch.tensor with dims (Nz,fy,fx)

        Note the torch.fft library uses dtype=torch.complex64
        This may be an issue for GPU implementation

        """

        if self.transform_mode == "standard":
            Ein = Ein.float()
            Ein -= torch.mean(Ein)
            Ein /= torch.std(Ein)

        elif self.transform_mode == "min-max":
            Ein = Ein.float()
            Ein -= torch.min(Ein)
            Ein /= torch.max(Ein)

        Etfft = torch.fft.fft2(Ein)
        Eofft = Etfft*torch.exp(1j*2*np.pi*z_tnsr/self.lam *
                                torch.sqrt(1-self.lam**2*(self.fx**2+self.fy**2)))

        # It might be helpful if we could omit this step.  It would save an inverse fft.
        Eout = torch.fft.ifft2(Eofft)
        return Eout
    
    
    def create_mapping(self):
        """
        Create map from tile coordinates (x,y) to indices to slice in image to extract that tile. 
        """
        self.idx2slice = {}
        for row_idx in range(self.Nx//self.step_size):

            if row_idx*self.step_size+self.tile_size > self.Nx:
                image_pixel_x = self.Nx-self.tile_size
                row_slice = slice(-self.tile_size, None)
                row_break = True
            else:
                image_pixel_x = row_idx*self.step_size
                row_slice = slice(row_idx*self.step_size,
                                  row_idx*self.step_size+self.tile_size)
                row_break = False

            for col_idx in range(self.Ny//self.step_size):

                if col_idx*self.step_size+self.tile_size > self.Ny:
                    image_pixel_y = self.Ny-self.tile_size
                    col_slice = slice(-self.tile_size, None)
                    col_break = True
                else:
                    image_pixel_y = col_idx*self.step_size
                    col_slice = slice(col_idx*self.step_size,
                                      col_idx*self.step_size+self.tile_size)
                    col_break = False

                self.idx2slice[row_idx, col_idx] = (row_slice, col_slice)

                if col_break:
                    break

            if row_break:
                break
                
                
                
    def collate_image(self, idx_dict, image = None, mask = None):
        x, y = zip(*[(image[:, row_slice, col_slice], mask[row_slice, col_slice]) for ((row_idx, col_idx, row_slice, col_slice)) in idx_dict])


In [75]:
def create_mask(prop, h_idx, z_idx, z_ref):

    hid = h_idx + 1
    hid_mask = prop.h_ds["hid"] == hid

    # Filter particles based on h_idx
    x_part = prop.h_ds["x"].values[hid_mask]
    y_part = prop.h_ds["y"].values[hid_mask]
    z_part = prop.h_ds["z"].values[hid_mask]
    d_part = prop.h_ds["d"].values[hid_mask]

    z_indices = np.digitize(z_part, prop.z_bins) - 1
    # Initialize the UNET mask
    unet_mask = np.zeros((prop.x_arr.shape[0], prop.y_arr.shape[0]))
    z_mask = np.zeros((prop.x_arr.shape[0], prop.y_arr.shape[0]))
    num_particles = 0 
    
    if z_idx in z_indices:
        cond = np.where(z_idx == z_indices)
        x_part = x_part[cond]
        y_part = y_part[cond]
        z_part = z_part[cond]
        d_part = d_part[cond]
        
        #print(x_part, y_part, z_part, d_part)
        
        # Build the UNET mask using vectorized operations
        for part_idx in range(len(cond[0])):
            y_diff = (prop.y_arr[None, :] * 1e6 - y_part[part_idx])
            x_diff = (prop.x_arr[:, None] * 1e6 - x_part[part_idx])
            d_squared = (d_part[part_idx] / 2)**2
            unet_mask += ((y_diff**2 + x_diff**2) < d_squared).astype(float)
            z_diff = (z_part - z_ref)
            for particle in z_diff:
                z_mask += ((y_diff**2 + x_diff**2) < d_squared).astype(float) * particle
                num_particles += 1

    return torch.from_numpy(unet_mask).unsqueeze(0), num_particles, torch.from_numpy(z_mask).unsqueeze(0)


In [76]:
class LoadHolograms(Dataset):
    def __init__(
        self,
        file_path,
        n_bins=1000,
        shuffle=False,
        device="cpu",
        transform=False,
        lookahead=0,
        step_size=32,
        tile_size=32,
        balance=True,
    ):

        # num of waveprop windows
        self.n_bins = n_bins
        # device used
        self.device = device
        # shuffle frames
        self.shuffle = shuffle
        # num of frames to look ahead
        self.lookahead = lookahead
        # wavepropagator object on device
        self.propagator = WavePropagator(
            file_path,
            n_bins=n_bins,
            device=device,
            step_size=step_size,
            tile_size=tile_size,
        )
        self.transform = transform
        self.indices = [
            (x, y)
            for x in self.propagator.h_ds.hologram_number
            for y in range(self.n_bins - self.lookahead)
        ]

        self.tile_size = tile_size
        self.idx2slice = self.propagator.idx2slice
        self.balance = balance

    def __len__(self):
        if self.balance:
            return len(self.propagator.h_ds.hologram_number) * len(self.indices)
        return len(self.indices) * len(self.idx2slice)

    def __getitem__(self, idx):

        if self.balance:
            image_lst = []
            while len(image_lst) == 0:
                # flip coin
                infocus, outfocus = random.choice([[0, 1], [1, 0]])
                image_lst, particle_unet_labels_lst, particle_in_focus_lst = self.get_reconstructed_sub_images(
                    idx, part_per_holo=infocus, empt_per_holo=outfocus
                )
            return image_lst[0], particle_unet_labels_lst[0].squeeze(0), particle_in_focus_lst

        else:
            return self.random_batcher(idx)

    def random_batcher(self, idx):

        if self.shuffle:
            idx = random.choice(range(self.__len__()))

        # hologram_idx = idx // self.n_bins
        # plane_idx = idx // len(self.propagator.h_ds.hologram_number)
        hologram_idx, plane_idx = self.indices[(idx) // len(self.idx2slice)]
        z_props = self.propagator.z_centers[plane_idx : plane_idx + self.lookahead + 1]
        # z_props -= (z_props[1] - z_props[0]) / 2
        plane_indices = np.arange(plane_idx, plane_idx + self.lookahead + 1)
        # select hologram
        image = (
            self.propagator.h_ds["image"]
            .isel(hologram_number=hologram_idx)
            .values.astype(float)
        )

        im = {
            "image": np.expand_dims(image, 0),
            "horizontal_flip": False,
            "vertical_flip": False,
        }

        # add transformations here
        if self.transform:
            for image_transform in self.transform:
                im = image_transform(im)
        image = im["image"]

        # make tensors of size lookahead + 1, and then add tensors
        prop_synths = torch.empty((len(z_props), image.shape[-2], image.shape[-1]))
        prop_phases = torch.empty((len(z_props), image.shape[-2], image.shape[-1]))
        masks = torch.empty((1, image.shape[-2], image.shape[-1]))
        z_masks = torch.empty((1, image.shape[-2], image.shape[-1]))

        for k, (z_prop, z_ind) in enumerate(zip(z_props, plane_indices)):
            image_prop = self.propagator.torch_holo_set(
                image.to(self.device),
                torch.FloatTensor([z_prop * 1e-6]).to(self.device),
            )
            # ABS (x-input)
            prop_synth = torch.abs(image_prop).squeeze(0)
            prop_synths[k] = prop_synth
            # Phase (x-input)
            prop_phase = torch.angle(image_prop).squeeze(0)
            prop_phases[k] = prop_phase
            # Mask (y-label)
            if k == 0:
                mask, num_particles, z_mask = create_mask(
                    self.propagator, hologram_idx, z_ind, z_prop
                )
                if im["horizontal_flip"]:
                    mask = torch.flip(mask, [1])
                    z_mask = torch.flip(z_mask, [1])
                if im["vertical_flip"]:
                    mask = torch.flip(mask, [2])
                    z_mask = torch.flip(z_mask, [2])
                masks[k] = mask
                z_masks[k] = z_mask

        # z_masks_window /= (z_props[-1] - z_props[0])

        # cat images and masks in color dim (0 since batch not added yet)
        image_stack = torch.cat([prop_synths, prop_phases], dim=0)

        # get tiles, slicing along coords in idx2slice array that maps coord position to slice range
        slice_coords = self.idx2slice[list(self.idx2slice)[idx % len(self.idx2slice)]]
        image_stack = image_stack[:, slice_coords[0], slice_coords[1]]
        masks = masks[slice_coords[0], slice_coords[1]]

        return (image_stack, masks)

    def get_full_plane(self, h_idx, z_idx_list, device = "cpu"):
        image = self.propagator.h_ds['image'].isel(hologram_number=h_idx+1).values
        # select hologram
        im = {
            "image": np.expand_dims(image, 0),
            "horizontal_flip": False,
            "vertical_flip": False
        }

        # add transformations here
        if self.transform:
            for image_transform in self.transform:
                im = image_transform(im)
        image = im["image"]
        
        for z_idx in z_idx_list:
            z_props = self.propagator.z_centers[z_idx: z_idx + self.lookahead + 1]
            # select planes for lookahead
            plane_indices = np.arange(z_idx, z_idx + self.lookahead + 1)

            # Make tensors of size lookahead + 1, and then add tensors
            prop_synths = np.zeros((lookahead + 1, *image.shape))
            prop_phases = np.zeros((lookahead + 1, *image.shape))
            masks = np.zeros((1, *image.shape[-2:]))
            particles_in_frames = []

            for k, (z_prop, z_ind) in enumerate(zip(z_props, plane_indices)):
                image_prop = self.propagator.torch_holo_set(
                    image.to(device),
                    torch.FloatTensor([z_prop*1e-6]).to(device)
                )
                # ABS (x-input)
                prop_synth = torch.abs(image_prop)
                prop_synths[k] = prop_synth
                # Phase (x-input)
                prop_phase = torch.angle(image_prop)
                prop_phases[k] = prop_phase  
                # Mask (y-label)
                if k == 0:
                    mask, num_particles, _ = create_mask(self.propagator, h_idx, z_ind, z_prop)
                    if im["horizontal_flip"]:
                        mask = torch.flip(mask, dims=(1,))
                    if im["vertical_flip"]:
                        mask = torch.flip(mask, dims=(2,))
                    masks[k] = mask
                    particles_in_frames.append(num_particles)

            # cat target frames with lookahead context frames, convert to ndarrays
            synth_window = np.concatenate(prop_synths, axis=0)
            phases_window = np.concatenate(prop_phases, axis=0)
            masks_window = np.concatenate(masks, axis=0)
            
            # cat images and masks in color dim (0 since batch not added yet)
            image_stack = np.concatenate([synth_window, phases_window], axis=0)
            masks_stack = np.concatenate([masks_window], axis=0)

            yield (torch.from_numpy(image_stack).float(), torch.from_numpy(masks_stack).int())
        
        
        # given a full image, slice coordinates defined in slice.idx2slice, return dict of tiles {(x,y): tensor}
    def sequential_tile(self, full_plane):
        tiles_dict = defaultdict()
        for slice_coords in self.idx2slice:
            tiles_dict[slice_coords] = full_plane[:,self.idx2slice[slice_coords][0], self.idx2slice[slice_coords][1]]
        return tiles_dict
    
        # given dict of tiles {(x,y): tensor tile} (as returned from sequential_tile()), reconstruct full image
        # stride overlap is sum of both tiles
    def tile_reconstruct(self, tile_dict, largex, largey):
        template = torch.zeros(1, 1, largex, largey)
        counter = torch.zeros(largex, largey)
        #template = torch.zeros(1,tile_dict[list(tile_dict.keys())[0]].shape[1],largex, largey)
        for coords in tile_dict:
            tile = tile_dict[coords]
            slice_coords = self.idx2slice[coords]
            template[:, :, slice_coords[0], slice_coords[1]] += tile
            counter[slice_coords[0], slice_coords[1]] += 1
        return template, counter
            
        # given image index, model, loss function, device, get full images at idx, split in to tiles, and get tile-wise prediction from model
        # attach back together, and then evaluate loss between full prediction image and fullm ask image
        
        
    def get_full_plane(self, h_idx, z_idx_list, device = "cpu"):
        image = self.propagator.h_ds['image'].isel(hologram_number=h_idx+1).values
        # select hologram
        im = {
            "image": np.expand_dims(image, 0),
            "horizontal_flip": False,
            "vertical_flip": False
        }
        
        if self.transform:
            for image_transform in self.transform:
                im = image_transform(im)
        image = im["image"]
        
        for z_idx in z_idx_list:
            z_props = self.propagator.z_centers[z_idx: z_idx + self.lookahead + 1]
            # select planes for lookahead
            plane_indices = np.arange(z_idx, z_idx + self.lookahead + 1)

            # Make tensors of size lookahead + 1, and then add tensors
            prop_synths = np.zeros((self.lookahead + 1, *image.shape))
            prop_phases = np.zeros((self.lookahead + 1, *image.shape))
            masks = np.zeros((1, *image.shape[-2:]))
            particles_in_frames = []

            for k, (z_prop, z_ind) in enumerate(zip(z_props, plane_indices)):
                image_prop = self.propagator.torch_holo_set(
                    image.to(device),
                    torch.FloatTensor([z_prop*1e-6]).to(device)
                )
                # ABS (x-input)
                prop_synth = torch.abs(image_prop)
                prop_synths[k] = prop_synth
                # Phase (x-input)
                prop_phase = torch.angle(image_prop)
                prop_phases[k] = prop_phase  
                # Mask (y-label)
                if k == 0:
                    mask, num_particles, _ = create_mask(self.propagator, h_idx, z_ind, z_prop)
                    if im["horizontal_flip"]:
                        mask = torch.flip(mask, dims=(1,))
                    if im["vertical_flip"]:
                        mask = torch.flip(mask, dims=(2,))
                    masks[k] = mask
                    particles_in_frames.append(num_particles)

            # cat target frames with lookahead context frames, convert to ndarrays
            synth_window = np.concatenate(prop_synths, axis=0)
            phases_window = np.concatenate(prop_phases, axis=0)
            masks_window = np.concatenate(masks, axis=0)
            
            # cat images and masks in color dim (0 since batch not added yet)
            image_stack = np.concatenate([synth_window, phases_window], axis=0)
            masks_stack = np.concatenate([masks_window], axis=0)

            yield (torch.from_numpy(image_stack).float(), torch.from_numpy(masks_stack).int())
        
        
    def full_inference(self, idx, model, loss = "dice", device = "cpu"):
        full_image, full_mask = self.get_full_plane(idx, device = device)
        image_tiles = self.sequential_tile(full_image)
        mask_tiles = self.sequential_tile(full_mask)
        with torch.no_grad():
            for coords in image_tiles:
                image_tiles[coords] = model(image_tiles[coords].unsqueeze(0))[:,0:1,:,:]
        full_inference_frame = self.tile_reconstruct(image_tiles)[0].squeeze(0)[0:1,:,:].float()
        full_inference_mask, full_mask_counter = self.tile_reconstruct(mask_tiles)
        test_criterion = load_loss(loss, split = "validation")
        val_loss = test_criterion(full_inference_frame, full_inference_mask)
        return(val_loss, full_inference_frame, full_mask_counter)
            
    def get_reconstructed_sub_images(
        self, h_idx, part_per_holo=None, empt_per_holo=None
    ):
        """
        Reconstruct a hologram at specific planes to provide training data
        with a specified number of sub images containing and not containing
        particles
        """
        h_idx = h_idx // len(self.indices)
        self.step_size = self.propagator.step_size
        self.tile_size = self.propagator.tile_size
        step_size = self.step_size
        tile_size = self.tile_size

        #### roughly half of the empty cases should be near in focus ####
        empt_near_cnt = empt_per_holo // 2
        ####

        # locate particle information corresponding to this hologram
        self.h_ds = self.propagator.h_ds
        particle_idx = np.where(self.h_ds["hid"].values == h_idx + 1)

        x_part = self.h_ds["x"].values[particle_idx]
        y_part = self.h_ds["y"].values[particle_idx]
        z_part = self.h_ds["z"].values[particle_idx]
        # not used but here it is
        d_part = self.h_ds["d"].values[particle_idx]

        # create a 3D histogram
        in_data = np.stack((x_part, y_part, z_part)).T
        h_part = np.histogramdd(
            in_data,
            bins=[
                self.propagator.tile_x_bins,
                self.propagator.tile_y_bins,
                self.propagator.z_bins,
            ],
        )[0]
        # specify the z bin locations of the particles
        z_part_bin_idx = np.digitize(z_part, self.propagator.z_bins) - 1

        # smoothing kernel accounts for overlapping subimages when the
        # subimage is larger than the stride
        ratio = self.tile_size // self.step_size
        if self.step_size < self.tile_size:
            overlap_kernel = np.ones((ratio, ratio))
            for z_idx in range(h_part.shape[-1]):
                h_part[:, :, z_idx] = convolve2d(h_part[:, :, z_idx], overlap_kernel)[
                    ratio - 1 : h_part.shape[0] + ratio - 1,
                    ratio - 1 : h_part.shape[1] + ratio - 1,
                ]

        # locate all the cases where particles are and are not present
        # to sample from those cases
        if self.step_size < self.tile_size:
            # note that the last bin is ommitted from each to avoid edge cases where
            # the image is not complete

            edge_idx = ratio - 1

            # find the locations where particles are in focus
            loc_idx = np.where(h_part[:-edge_idx, :-edge_idx, :] > 0)
            # find locations where particles are not in focus
            empt_idx = np.where(h_part[:-edge_idx, :-edge_idx, :] == 0)
            #### find locations where particles are nearly in focus  ####
            zdiff = np.diff(h_part[:-edge_idx, :-edge_idx, :], axis=2)
            zero_pad = np.zeros(h_part[:-edge_idx, :-edge_idx, :].shape[0:2] + (1,))
            near_empt_idx = np.where(
                (h_part[:-edge_idx, :-edge_idx, :] == 0)
                & (
                    (np.concatenate([zdiff, zero_pad], axis=2) == 1)
                    | (np.concatenate([zero_pad, zdiff], axis=2) == -1)
                )
            )
            ####
        else:
            # find the locations where particles are in focus
            loc_idx = np.where(h_part > 0)
            # find locations where particles are not in focus
            empt_idx = np.where(h_part == 0)
            #### find locations where particles are nearly in focus ####
            zdiff = np.diff(h_part, axis=2)
            zero_pad = np.zeros(h_part.shape[0:2] + (1,))
            near_empt_idx = np.where(
                (h_part == 0)
                & (
                    (np.concatenate([zdiff, zero_pad], axis=2) == 1)
                    | (np.concatenate([zero_pad, zdiff], axis=2) == -1)
                )
            )
            ####
            
        loc_x_idx = loc_idx[0]
        loc_y_idx = loc_idx[1]
        loc_z_idx = loc_idx[2]

        # select sub images with particles in them
        if part_per_holo > loc_idx[0].size:
            pass
            # pick the entire set
            
        else:
            # randomly select particles from the set
            sel_part_idx = np.random.choice(
                np.arange(loc_x_idx.size, dtype=int), size=part_per_holo, replace=False
            )
            loc_x_idx = loc_x_idx[sel_part_idx]
            loc_y_idx = loc_y_idx[sel_part_idx]
            loc_z_idx = loc_z_idx[sel_part_idx]

        # randomly select empties from the empty set
        #### Add nearly in focus cases to the training data ####
        sel_empt_idx = np.random.choice(
            np.arange(near_empt_idx[0].size, dtype=int),
            size=empt_near_cnt,
            replace=False,
        )  # select nearly in focus cases
        
        ####
        sel_empt_idx = np.concatenate(
            [
                np.random.choice(
                    np.arange(empt_idx[0].size, dtype=int),
                    size=(empt_per_holo - empt_near_cnt),
                    replace=False,
                ),
                sel_empt_idx,
            ]
        )  # select random out of focus cases
        empt_x_idx = empt_idx[0][sel_empt_idx]
        empt_y_idx = empt_idx[1][sel_empt_idx]
        empt_z_idx = empt_idx[2][sel_empt_idx]

        # full set of plane indices to reconstruct (empty and with particles)
        z_full_idx = np.unique(np.concatenate((loc_z_idx, empt_z_idx)))
        _z_full_idx = []
        for zp in z_full_idx:
            if zp >= (self.n_bins-self.lookahead):
                _z_full_idx.append(zp - self.lookahead - 1)
            else:
                _z_full_idx.append(zp)
        z_full_idx = np.array(_z_full_idx)

        # build the torch tensor for reconstruction
        z_plane = (
            torch.tensor(
                self.propagator.z_centers[z_full_idx] * 1e-6, device=self.device
            )
            .unsqueeze(-1)
            .unsqueeze(-1)
        )

        # grab the sub images corresponding to the selected data points
        particle_in_focus_lst = []  # training labels for if particle is in focus
        particle_unet_labels_lst = []  # training labels for if particle is in focus
        image_lst = []  # sliced reconstructed image
        image_index_lst = []  # indices used to identify the image slice
        image_corner_coords = []  # coordinates of the corner of the image slice

        for sub_idx, z_idx in enumerate(z_full_idx):
            z_props = self.propagator.z_centers[z_idx : z_idx + self.lookahead + 1]
            plane_indices = np.arange(z_idx, z_idx + self.lookahead + 1)
            # select hologram
            image = (
                self.propagator.h_ds["image"]
                .isel(hologram_number=h_idx)
                .values.astype(float)
            )

            im = {
                "image": np.expand_dims(image, 0),
                "horizontal_flip": False,
                "vertical_flip": False,
            }

            # add transformations here
            if self.transform:
                for image_transform in self.transform:
                    im = image_transform(im)
            image = im["image"]

            # make tensors of size lookahead + 1, and then add tensors
            prop_synths = torch.empty((len(z_props), image.shape[-2], image.shape[-1]))
            prop_phases = torch.empty((len(z_props), image.shape[-2], image.shape[-1]))
            masks = torch.empty((1, image.shape[-2], image.shape[-1]))
            z_masks = torch.empty((1, image.shape[-2], image.shape[-1]))

            for k, (z_prop, z_ind) in enumerate(zip(z_props, plane_indices)):
                image_prop = self.propagator.torch_holo_set(
                    image.to(self.device),
                    torch.FloatTensor([z_prop * 1e-6]).to(self.device),
                )
                # ABS (x-input)
                prop_synth = torch.abs(image_prop).squeeze(0)
                prop_synths[k] = prop_synth
                # Phase (x-input)
                prop_phase = torch.angle(image_prop).squeeze(0)
                prop_phases[k] = prop_phase
                # Mask (y-label)
                if k == 0:
                    mask, num_particles, z_mask = create_mask(
                        self.propagator, h_idx, z_ind, z_prop
                    )
                    if im["horizontal_flip"]:
                        mask = torch.flip(mask, [1])
                        z_mask = torch.flip(z_mask, [1])
                    if im["vertical_flip"]:
                        mask = torch.flip(mask, [2])
                        z_mask = torch.flip(z_mask, [2])
                    masks[k] = mask
                    z_masks[k] = z_mask
            # z_masks_window /= (z_props[-1] - z_props[0])

            # cat images and masks in color dim (0 since batch not added yet)
            image_stack = torch.cat([prop_synths, prop_phases], dim=0)

            # Build the mask and pick out
            part_set_idx = np.where(loc_z_idx == z_idx)[0]
            empt_set_idx = np.where(empt_z_idx == z_idx)[0]

            # initialize the UNET mask
            # locate all particles in this plane
            part_in_plane_idx = np.where(z_part_bin_idx == z_idx)[0] 

            for part_idx in part_set_idx:
                x_idx = loc_x_idx[part_idx]
                y_idx = loc_y_idx[part_idx]
                image_lst.append(
                    image_stack[
                        :,
                        x_idx * step_size : (x_idx * step_size + tile_size),
                        y_idx * step_size : (y_idx * step_size + tile_size),
                    ]
                )
                image_index_lst.append([x_idx, y_idx, z_idx])
                image_corner_coords.append(
                    [
                        self.propagator.x_arr[x_idx * step_size],
                        self.propagator.y_arr[y_idx * step_size],
                    ]
                )
                particle_in_focus_lst.append(1)
                particle_unet_labels_lst.append(
                    masks[
                        :,
                        x_idx * step_size : (x_idx * step_size + tile_size),
                        y_idx * step_size : (y_idx * step_size + tile_size),
                    ]
                )

            for empt_idx in empt_set_idx:
                x_idx = empt_x_idx[empt_idx]
                y_idx = empt_y_idx[empt_idx]
                image_lst.append(
                    image_stack[
                        :,
                        x_idx * step_size : (x_idx * step_size + tile_size),
                        y_idx * step_size : (y_idx * step_size + tile_size),
                    ]
                )
                image_index_lst.append([x_idx, y_idx, z_idx])
                image_corner_coords.append(
                    [
                        self.propagator.x_arr[x_idx * step_size],
                        self.propagator.y_arr[y_idx * step_size],
                    ]
                )
                particle_in_focus_lst.append(0)
                particle_unet_labels_lst.append(
                    masks[
                        :,
                        x_idx * step_size : (x_idx * step_size + tile_size),
                        y_idx * step_size : (y_idx * step_size + tile_size),
                    ]
                )
                
        #print("Returning", len(image_lst))
            
        return image_lst, particle_unet_labels_lst, particle_in_focus_lst


## Visualizations

In [77]:
torch.cuda.empty_cache()
conf_file_path = "../results/batch1/model.yml"
#conf_file_path = "/glade/work/schreck/repos/HOLO/style/holodec-ml/results/manopt/model.yml"
# load in from config file
with open(conf_file_path) as cf:
    conf = yaml.load(cf, Loader=yaml.FullLoader)

        
# edit color channels to be compatible with lookahead
#conf["model"]["in_channels"] = 2 * (conf["data"]["lookahead"] + 1)
conf["model"]["in_channels"] = 1
# load transformations from config
if "Normalize" in conf["transforms"]["training"]:
    conf["transforms"]["validation"]["Normalize"]["mode"] = conf["transforms"][
        "training"
    ]["Normalize"]["mode"]
    conf["transforms"]["inference"]["Normalize"]["mode"] = conf["transforms"][
        "training"
    ]["Normalize"]["mode"]

train_transforms = LoadTransformations(conf["transforms"]["training"])
valid_transforms = LoadTransformations(conf["transforms"]["validation"])

# load lookahead depth from config
lookahead = int(conf["data"]["lookahead"])
tile_size = int(conf["data"]["tile_size"])
step_size = int(conf["data"]["step_size"])
# Set up CUDA/CPU devices
is_cuda = torch.cuda.is_available()
data_device = (
    torch.device("cpu") if "device" not in conf["data"] else conf["data"]["device"]
)

if torch.cuda.device_count() >= 2 and "cuda" in data_device:
    data_device = "cuda:0"
    device = "cuda:1"
    device_ids = list(range(1, torch.cuda.device_count()))
else:
    data_device = torch.device("cpu")
    device = (
        torch.device(torch.cuda.current_device())
        if is_cuda
        else torch.device("cpu")
    )
    device_ids = list(range(torch.cuda.device_count()))
logging.info(f"There are {torch.cuda.device_count()} GPUs available")
logging.info(
    f"Using device {data_device} to perform wave propagation, and {device_ids} for training the model"
)


# load tiling size
step_size = int(conf["data"]["step_size"])


# hard code shuffle, n_bins
shuffle =False
n_bins = 1000


In [78]:
# get data

train_dataset = LoadHolograms("/glade/p/cisl/aiml/ai4ess_hackathon/holodec/synthetic_holograms_500particle_gamma_4872x3248_training.nc", shuffle = False, device = data_device, 
                              n_bins = n_bins, transform = train_transforms, lookahead = lookahead, tile_size = tile_size, step_size = step_size, balance = True)

test_dataset = LoadHolograms("/glade/p/cisl/aiml/ai4ess_hackathon/holodec/synthetic_holograms_500particle_gamma_4872x3248_validation.nc", shuffle = False, device = data_device, 
                             n_bins = n_bins, transform = valid_transforms, lookahead = lookahead, tile_size = tile_size, step_size = step_size, balance = True)


In [79]:
inputs = train_dataset.__getitem__(200)

In [ ]:
inputs[0].shape

In [ ]:
h_idx = 1
z_idx_list = [0,1,2,3]
gener = train_dataset.get_full_plane(h_idx, z_idx_list, device = "cpu")
for (image_stack, masks_stack) in gener:
    print(image_stack.shape)


In [ ]:
loss, mask_frames, counter_frames = train_dataset.full_inference(5, [0,1,2,3], model)

In [ ]:
print(loss)

In [ ]:
# check loading
randint = 1040529
rand_img_raw, rand_masks_raw = train_dataset.__getitem__(randint)

while (int(torch.count_nonzero(rand_masks_raw[0])) < 30):
    randint = random.randint(0,train_dataset.__len__())
    rand_img_raw, rand_masks_raw = train_dataset.__getitem__(randint)
print(randint, int(torch.count_nonzero(rand_masks_raw[0])))
print(len(torch.nonzero(rand_masks_raw[0], as_tuple = False)))
print()

rand_img_raw, rand_masks_raw = train_dataset.__getitem__(randint)
image = rand_img_raw
print(len(image))

fontsize = 10
single_column_width = 10
plt.figure(figsize = (single_column_width/30, single_column_width / 1.61803398875))
fig, ax = plt.subplots(1, 4, figsize=(single_column_width, single_column_width * 1.61803398875), 
                         gridspec_kw={'hspace': 0.0, 'wspace': 0.2})
ax = ax.flatten()
ax[0].imshow(image[0], origin ='lower')
ax[1].imshow(image[1], origin = 'lower')
ax[2].imshow(image[2], origin = 'lower')
ax[3].imshow(image[3], origin = 'lower')

In [ ]:
image = counter_frames
fontsize = 10
single_column_width = 10
plt.figure(figsize = (single_column_width/30, single_column_width / 1.61803398875))
fig, ax = plt.subplots(1, 4, figsize=(single_column_width, single_column_width * 1.61803398875), 
                         gridspec_kw={'hspace': 0.0, 'wspace': 0.2})
ax = ax.flatten()
ax[0].imshow(image[0], origin ='lower')
ax[1].imshow(image[1], origin = 'lower')
ax[2].imshow(image[2], origin = 'lower')
ax[3].imshow(image[3], origin = 'lower')

In [ ]:
image = masks

fontsize = 10
single_column_width = 10
plt.figure(figsize = (single_column_width/30, single_column_width / 1.61803398875))
fig, ax = plt.subplots(1, 1, figsize=(single_column_width, single_column_width * 1.61803398875), 
                         gridspec_kw={'hspace': 0.0, 'wspace': 0.2})

ax.imshow(image[0], origin ='lower')



In [ ]:
%%time
val_loss = train_dataset.full_inference(randint, model)

In [ ]:
print(val_loss[0])

In [ ]:
image = val_loss[1]

fontsize = 10
single_column_width = 10
plt.figure(figsize = (single_column_width/30, single_column_width / 1.61803398875))
fig, ax = plt.subplots(1, 1, figsize=(single_column_width, single_column_width * 1.61803398875), 
                         gridspec_kw={'hspace': 0.0, 'wspace': 0.2})

ax.imshow(image[0] > 0.2, origin ='lower')

In [ ]:
image = val_loss[2]

fontsize = 10
single_column_width = 10
plt.figure(figsize = (single_column_width/30, single_column_width / 1.61803398875))
fig, ax = plt.subplots(1, 1, figsize=(single_column_width, single_column_width * 1.61803398875), 
                         gridspec_kw={'hspace': 0.0, 'wspace': 0.2})

ax.imshow(image, origin ='lower')

In [ ]:
mask_tiles = train_dataset.sequential_tile(masks)
print(mask_tiles[(0,0)].shape)

In [ ]:
mask_reconstruct_raw, mask_reconstruct_counter = train_dataset.tile_reconstruct(mask_tiles)
print(mask_reconstruct_counter.shape)

In [ ]:
image = mask_reconstruct_raw.squeeze(0).squeeze()

fontsize = 10
single_column_width = 10
plt.figure(figsize = (single_column_width/30, single_column_width / 1.61803398875))
fig, ax = plt.subplots(1, 1, figsize=(single_column_width, single_column_width * 1.61803398875), 
                         gridspec_kw={'hspace': 0.0, 'wspace': 0.2})

ax.imshow(image, origin ='lower')

In [ ]:
|